# Extracting data from Tableau extract to Pandas dataframe with Tableau Hyper API

# Download of Tableau Hyper APIs

In this workbook I used the Hyper API 0.0.16123 for Python that you can download from [this page](https://www.tableau.com/support/releases/hyper-api/0.0.16123).

# Setting up environment

I advice you to make a Python virtual environment and then install the API in it. You can follow [this guide](https://help.tableau.com/current/api/hyper_api/en-us/docs/hyper_api_installing.html) to learn how to install the downloaded files.

## Modules

In [1]:
import shutil
from pathlib import Path
import pandas as pd
from tableauhyperapi import HyperProcess, Telemetry, Connection, TableName, HyperException

## Variables

In [2]:
hyperPath='02.Unpackaged Tableau workbooks/Data/Datasources/salaries-by-college-type (1) (salaries-by-college-type (1)).hyper'
workingDir='03.Working folder'
workingFile=workingDir+'/working.hyper'

## Functions

In [3]:
def denormalizeHyper(src,dst):
    path_to_database = Path(shutil.copy(src=src,dst=dst)).resolve()
    return "File created: %s" % (workingFile)

In [4]:
def getSchemaNames(database):
    with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU) as hyper:
        with Connection(endpoint=hyper.endpoint,database=database) as connection:
            schema_names= connection.catalog.get_schema_names()
    return schema_names

In [5]:
def getTableNames(database,schema):
    with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU) as hyper:
        with Connection(endpoint=hyper.endpoint,database=database) as connection:
            table_names = connection.catalog.get_table_names(schema=schema)
    return table_names

In [6]:
def rowsInTable(database,schema,table):
    with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU) as hyper:
        with Connection(endpoint=hyper.endpoint,database=database) as connection:
            table_name = TableName(schema, table)
            rows_in_table = connection.execute_list_query(query=f"SELECT * FROM {table_name}")
    return rows_in_table

In [7]:
def rowsToPandas(rows_in_table):
    dfRows = pd.DataFrame(rows_in_table)
    return dfRows

In [8]:
def columnNamesPrint(database,schema,table):
    with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU) as hyper:
        with Connection(endpoint=hyper.endpoint,database=database) as connection:
            table_name = TableName(schema, table)
            table_definition = connection.catalog.get_table_definition(name=table_name)
            print("List of columns of %s table:" % (table))
            for column in table_definition.columns:
                print(f'''- {column.name} 
    type={column.type} and nullability={column.nullability}''')

In [9]:
def columnNamesList(database,schema,table):
    with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU) as hyper:
        with Connection(endpoint=hyper.endpoint,database=database) as connection:
            table_name = TableName(schema, table)
            table_definition = connection.catalog.get_table_definition(name=table_name)
            col=[]
            for column in table_definition.columns:
                col.append(column.name.unescaped)
    return col

## Runnung extraction

First of all, we have to denormalize data.

In [10]:
denormalizeHyper(hyperPath,workingFile)

'File created: 03.Working folder/working.hyper'

Since this point, we will go further using the _working.hyper_ file and not the original as it is normalized.

next, we wanna investigate the _schemas_ included in the extract

In [11]:
getSchemaNames(workingFile)


[SchemaName('Extract'), SchemaName('public')]

and, for both of them, get a list of underlying tables

In [12]:
getTableNames(workingFile,'Extract')

[TableName('Extract', 'Extract')]

In [13]:
getTableNames(workingFile,'public')

[]

Easily, we can assume that the data of our interests are stored in the data with _schema=Extract_ and _table=Extract_, so we will query it using the _rowsInTable_ function

In [14]:
rows=rowsInTable(workingFile,'Extract','Extract')

At this point we are ready to move our data from the rows variable to a Pandas dataframe.

In [15]:
df=rowsToPandas(rows)

In [16]:
df.head()

,0,1,2,3,4,5,6,7
0,Massachusetts Institute of Technology (MIT),Engineering,72200,126000,76800,99200,168000,220000
1,"Polytechnic University of New York, Brooklyn",Engineering,62400,114000,66800,96000,143000,190000
2,Worcester Polytechnic Institute (WPI),Engineering,61000,114000,80000,91200,137000,180000
3,Carnegie Mellon University (CMU),Engineering,61800,111000,63300,80100,150000,209000
4,Rensselaer Polytechnic Institute (RPI),Engineering,61100,110000,71600,85500,140000,182000


As shown in the previous output we are still missing the column names, so we will extract them using the _columnNamesList_ function and then use the list to map the data frame.

In [17]:
columnNamesPrint(workingFile,'Extract','Extract')

List of columns of Extract table:
- "School Name" 
    type=TEXT and nullability=Nullability.NULLABLE
- "School Type" 
    type=TEXT and nullability=Nullability.NULLABLE
- "Starting Median Salary" 
    type=BIG_INT and nullability=Nullability.NULLABLE
- "Mid-Career Median Salary" 
    type=BIG_INT and nullability=Nullability.NULLABLE
- "Mid-Career 10th Percentile Salary" 
    type=BIG_INT and nullability=Nullability.NULLABLE
- "Mid-Career 25th Percentile Salary" 
    type=BIG_INT and nullability=Nullability.NULLABLE
- "Mid-Career 75th Percentile Salary" 
    type=BIG_INT and nullability=Nullability.NULLABLE
- "Mid-Career 90th Percentile Salary" 
    type=BIG_INT and nullability=Nullability.NULLABLE


In [18]:
col=columnNamesList(workingFile,'Extract','Extract')

In [19]:
col

['School Name',
 'School Type',
 'Starting Median Salary',
 'Mid-Career Median Salary',
 'Mid-Career 10th Percentile Salary',
 'Mid-Career 25th Percentile Salary',
 'Mid-Career 75th Percentile Salary',
 'Mid-Career 90th Percentile Salary']

In [20]:
df.columns=col

In [21]:
df.head()

,School Name,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Massachusetts Institute of Technology (MIT),Engineering,72200,126000,76800,99200,168000,220000
1,"Polytechnic University of New York, Brooklyn",Engineering,62400,114000,66800,96000,143000,190000
2,Worcester Polytechnic Institute (WPI),Engineering,61000,114000,80000,91200,137000,180000
3,Carnegie Mellon University (CMU),Engineering,61800,111000,63300,80100,150000,209000
4,Rensselaer Polytechnic Institute (RPI),Engineering,61100,110000,71600,85500,140000,182000
